In [ ]:
#Capa 1: Heurísticas tipo ModSecurity (no ML)

import pandas as pd
import numpy as np
import re
from collections import Counter

# ==============================
# Cargar dataset base
# ==============================
# Ajusta el path si es necesario
df = pd.read_csv('access_log_structured.csv')


## Investigación y Justificación de la Capa Heurística

### 1. Planteamiento del problema específico

El tráfico analizado en los registros de acceso presenta una alta proporción de peticiones automatizadas (bots, scrapers y spiders) caracterizadas por comportamientos persistentes y de baja intensidad. En este escenario, los mecanismos clásicos de rate limiting global resultan insuficientes, ya que el volumen total de peticiones no alcanza umbrales críticos a nivel del sistema, mientras que entidades individuales (IPs o User-Agents) exhiben patrones anómalos sostenidos en el tiempo.

El problema no consiste en detectar picos de tráfico, sino en identificar **desviaciones relativas de comportamiento por entidad**, lo cual justifica el uso de una capa heurística stateful basada en reglas.

---

### 2. Limitaciones del rate limiting global

El rate limiting tradicional aplica umbrales fijos de peticiones por unidad de tiempo a nivel global o por endpoint. Este enfoque presenta las siguientes limitaciones en el contexto de logs reales:

- No distingue entre tráfico humano y automatizado.
- Es vulnerable a ataques distribuidos de baja intensidad (*low-and-slow*).
- No captura patrones persistentes que solo son evidentes al analizar el historial por IP.
- Penaliza tráfico legítimo durante picos normales de uso.

Desde un punto de vista teórico, el rate limiting global opera sobre una **métrica absoluta**, mientras que el tráfico automatizado moderno se caracteriza por **anomalías relativas**.

---

### 3. Rate limiting adaptativo por IP y User-Agent

El enfoque propuesto redefine el rate limiting como un problema de **detección de outliers por entidad**, utilizando métricas estadísticas relativas:

- Frecuencia de peticiones por IP
- Proporción respecto a la media del conjunto
- Persistencia temporal
- Diversidad de recursos accedidos

Formalmente, una IP se considera anómala si:



requests(IP) > k × mean(requests(all IPs)), k > 1


Este criterio permite detectar *heavy hitters*, una técnica ampliamente utilizada en detección de anomalías de red y mitigación de ataques distribuidos.

**Ventaja teórica clave**:  
Este método es invariante al volumen total de tráfico y se adapta automáticamente a la carga normal del sistema.





### 4. Análisis comportamental vs inspección puntual

A diferencia de la inspección por petición individual, la capa heurística se basa en **agregación y memoria de estado**, lo cual permite:

- Identificar patrones que no son maliciosos de forma aislada.
- Penalizar comportamientos repetitivos y persistentes.
- Diferenciar entre errores ocasionales y exploración automatizada.

Esto convierte el sistema en un **detector de comportamiento**, no en un filtro sintáctico.

---

### 5. Uso del User-Agent como factor de score

El User-Agent no se emplea como una regla determinista, sino como una señal parcial dentro de un sistema de scoring:

- User-Agents asociados a herramientas automatizadas aumentan el score.
- User-Agents vacíos o genéricos se penalizan.
- La decisión final depende de la combinación con métricas de frecuencia y error.

Desde un punto de vista teórico, este enfoque reduce falsos positivos al evitar decisiones basadas en una única característica fácilmente falsificable.

---

In [ ]:
df_user_agent = pd.DataFrame(df['user_agent'].value_counts())
display(df_user_agent.head(5))
display(df_user_agent.columns)

In [ ]:
def is_bot(user_agent:str) -> bool:
  "Detecta si un user-agent pertenece a un bot/crawler/spider"
  if not user_agent:
    return False 
  user_agent = user_agent.lower()
  
  # Nombres de bots conocidos
  known_bots = {
    'googlebot', 
    'bingbot', 
    'yandexbot', 
    'applebot',
    'duckduckbot', 
    'baiduspider', 
    'sogou', 
    'bytespider',
    'amazonbot', 
    'gptbot', 
    'chatgpt-user', 
    'oai-searchbot',
    'claudebot', 
    'google-cloudvertexbot', 
    'google-extended',
    'perplexitybot', 
    'meta-externalagent', 
    'meta-webindexer',
    'tiktokspider', 
    'openai.com-bot', 
    'google.bot',
    # Poco comunes pero encontrados en el archivo de access log
    'thinkbot', 
    'petalbot'
    # No es bot pero se asumirá que sí debido a que los comportamientos no son permitidos
    'securitytxtresearch'
    #'SecurityTxtResearch'
  }

  # Verificar nombres de bots conocidos
  for bot in known_bots:
    if bot in user_agent:
      return True

  # Patrón: "dominio.com-bot" o "dominio.bot"
  import re
  pattern = r'[a-z0-9.-]+\.(?:com|org|net|io)[-.]bot'
  if re.search(pattern, user_agent):
    return True

  return False

In [ ]:
df['is_bot'] = df['user_agent'].apply(is_bot)

display(df.head())
display(df['is_bot'].value_counts())


# Crear máscara para los bots
bot_mask = df['is_bot'] == True
# Modificar la columna 'attack' para los bots 
df['anomaly_score_ua'] = bot_mask.astype(int)


### Contar requests por IP en ventanas cortas

In [ ]:
WINDOW = '30s'      # ventana corta
REQ_THRESHOLD = 20 # requests por ventana

ip_rate = (
    df
    .set_index('timestamp')
    .groupby('ip_client')
    .resample(WINDOW)
    .size()
    .reset_index(name='req_count')
)


### detectar ventanas anómalas

In [ ]:
ip_rate['is_high_rate_window'] = ip_rate['req_count'] > REQ_THRESHOLD

### agregar a nivel IP

In [ ]:
ip_rate_stats = ip_rate.groupby('ip_client').agg(
    high_rate_windows=('is_high_rate_window', 'sum'),
    max_requests_window=('req_count', 'max')
).reset_index()


### score heurístico por IP

In [ ]:

ip_rate_stats['rate_anomaly'] = ip_rate_stats['high_rate_windows'] >= 2


### Ahora fusionamos frecuencia total + error rate + rate temporal:

In [ ]:

ip_stats = df.groupby('ip_client').agg(
    total_requests=('ip_client', 'count'),
    unique_urls=('request', 'nunique'),
    error_rate=('status', lambda x: np.mean(x.astype(str).str.startswith(('4','5'))))
).reset_index()

ip_stats = ip_stats.merge(ip_rate_stats, on='ip_client', how='left')

ip_stats[['high_rate_windows','max_requests_window']] = (
    ip_stats[['high_rate_windows','max_requests_window']]
    .fillna(0)
)

mean_requests = ip_stats['total_requests'].mean()
ip_stats['high_rate'] = ip_stats['total_requests'] > 5 * mean_requests

ip_stats['anomaly_score_ip'] = (
    (ip_stats['high_rate_windows'] >= 2).astype(int) +
    (ip_stats['error_rate'] > 0.4).astype(int) +
    ip_stats[['high_rate']].sum(axis=1)
)

ip_stats['is_banned'] = ip_stats['anomaly_score_ip'] >= 2

df = df.merge(ip_stats[['ip_client','anomaly_score_ip','is_banned']], on='ip_client', how='left')



### Clasificación heurística final

In [ ]:

def classify(row):
    score = 0
    score += row.get('anomaly_score_ip',0)
    score += row.get('anomaly_score_ua',0)
    return 'Anomaly' if score >= 2 else 'Normal'

df['heuristic_label'] = df.apply(classify, axis=1)



### Sistemas stateful y reputación dinámica

El mantenimiento de estado por IP y User-Agent permite construir una reputación dinámica basada en el historial de comportamiento. Este enfoque es equivalente a un **sistema de reglas stateful**, donde las decisiones se basan en acumulación de evidencia.

Este modelo es especialmente eficaz contra:
- Scraping persistente
- Reconocimiento progresivo
- Ataques automatizados que evitan umbrales fijos

Teóricamente, la incorporación de memoria transforma reglas locales en un sistema de decisión globalmente consistente.


### Limpieza de columnas y resultados

In [ ]:

df_clean = df.drop(columns=['ip_client','user_agent','ident','referrer','is_bot'], errors='ignore')

print(df_clean['heuristic_label'].value_counts())



###  Relación con ataques reales

Las heurísticas implementadas permiten detectar:

- Scraping: alta frecuencia + diversidad de URLs
- Bots: patrones de User-Agent + ritmo constante
- Reconocimiento: alta tasa de errores HTTP
- DoS lento: persistencia por IP

Estas detecciones no requieren firmas explícitas, sino desviaciones de comportamiento.



### Respaldo teórico y referencias

- NIST SP 800-94 – Intrusion Detection and Prevention Systems  
  https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-94.pdf

- Sommer & Paxson – *Outside the Closed World*  
  https://ieeexplore.ieee.org/document/5504793

- ModSecurity Reference Manual  
  https://github.com/SpiderLabs/ModSecurity/wiki/Reference-Manual-(v2.x)

- Cloudflare – Adaptive Rate Limiting  
  https://www.cloudflare.com/ddos/

- Estan & Varghese – Heavy-Hitter Detection  
  https://dl.acm.org/doi/10.1145/316188.316214
